In [1]:
print("Kernel working fine!")

Kernel working fine!


## Adding basic imports

In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
829,Snakes on a Train is a movie I rented due to t...,negative
159,I think One True Thing is one of Meryl Streeps...,positive
869,"Fame, I think, was the best movie that I have ...",positive
135,This started out as a good sketch comedy. The ...,negative
678,"I was prepared for a bad movie, and a bad movi...",negative


## Data Pre-Processing

In [4]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
import nltk
nltk.download('wordnet')

df = normalize_text(df)
df.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/atharvabot7/nltk_data...


,review,sentiment
829,snake train movie rented due pure amusement th...,negative
159,think one true thing one meryl streep finest m...,positive
869,fame think best movie ever seen way funny dram...,positive
135,started good sketch comedy first show good loo...,negative
678,prepared bad movie bad movie guess complain tw...,negative


In [7]:
# Checking the number fo positive and negative sentiments for bias

df['sentiment'].value_counts()

sentiment
negative    257
positive    243
Name: count, dtype: int64

In [8]:
# Choosing only positive and negative sentiments, removing all others

x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'].value_counts()

sentiment
negative    257
positive    243
Name: count, dtype: int64

In [10]:
# Mapiing Positive sentiment to 1 and Negative to 0.

df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
829,snake train movie rented due pure amusement th...,0
159,think one true thing one meryl streep finest m...,1
869,fame think best movie ever seen way funny dram...,1
135,started good sketch comedy first show good loo...,0
678,prepared bad movie bad movie guess complain tw...,0


In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [18]:
vectorizer = CountVectorizer(max_features=80)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

## Train Test Split

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/atharvachundurwar841/MLOps-Project.mlflow')
dagshub.init(repo_owner='atharvachundurwar841', repo_name='MLOps-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")

Initialized MLflow to track repo "atharvachundurwar841/MLOps-Project"

Repository atharvachundurwar841/MLOps-Project initialized!

<Experiment: artifact_location='mlflow-artifacts:/57c5e8e707f94e83b4edbec0a500921b', creation_time=1755713107074, experiment_id='0', last_update_time=1755713107074, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [20]:
import mlflow
import logging
import os
import time
import tempfile
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 80)
        mlflow.log_param("test_size", 0.3)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model without using registry endpoints...")
        # --- SAFE PATH FOR DAGSHUB/REMOTE MLflow SERVERS ---
        # Save the model to a temp dir and log it as artifacts to avoid /logged-models endpoints
        from mlflow.sklearn import save_model

        with tempfile.TemporaryDirectory() as tmpdir:
            local_model_dir = os.path.join(tmpdir, "model")
            save_model(sk_model=model, path=local_model_dir)
            # This uploads the saved folder under the run's artifacts
            mlflow.log_artifacts(local_model_dir, artifact_path="model")
        # ----------------------------------------------------

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-08-20 11:23:18,368 - INFO - Starting MLflow run...
2025-08-20 11:23:18,651 - INFO - Logging preprocessing parameters...
2025-08-20 11:23:19,620 - INFO - Initializing Logistic Regression model...
2025-08-20 11:23:19,621 - INFO - Fitting the model...
2025-08-20 11:23:19,631 - INFO - Model training complete.
2025-08-20 11:23:19,632 - INFO - Logging model parameters...
2025-08-20 11:23:19,900 - INFO - Making predictions...
2025-08-20 11:23:19,903 - INFO - Calculating evaluation metrics...
2025-08-20 11:23:19,914 - INFO - Logging evaluation metrics...
2025-08-20 11:23:24,623 - INFO - Saving and logging the model without using registry endpoints...
2025-08-20 11:23:30,752 - INFO - Model training and logging completed in 12.10 seconds.
2025-08-20 11:23:30,754 - INFO - Accuracy: 0.7066666666666667
2025-08-20 11:23:30,754 - INFO - Precision: 0.7285714285714285
2025-08-20 11:23:30,755 - INFO - Recall: 0.6710526315789473
2025-08-20 11:23:30,755 - INFO - F1 Score: 0.6986301369863014


🏃 View run luxuriant-asp-991 at: https://dagshub.com/atharvachundurwar841/MLOps-Project.mlflow/#/experiments/0/runs/c8edfe44ba834fc896457886e19006de
🧪 View experiment at: https://dagshub.com/atharvachundurwar841/MLOps-Project.mlflow/#/experiments/0
